In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_csv('data/comp-merged.csv')
dates = df['date']
df.drop(columns=['date', 'diff', 'bias_coef'], axis=1, inplace=True)
df.head()

In [ ]:
X = df.drop(columns=['value_met'], axis=1)
y = df['value_met']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)#, random_state=42)

In [ ]:
svr_params = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

tuned_svr = GridSearchCV(SVR(), svr_params, cv=5, n_jobs=-1, verbose=1)
tuned_svr.fit(X_train, y_train)

print(tuned_svr.best_params_)
print(tuned_svr.best_score_)

In [ ]:
model = tuned_svr.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

In [ ]:
xgb_params = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.3, 0.5]
}

tuned_xgb = GridSearchCV(xgb.XGBRegressor(), xgb_params, cv=5, n_jobs=-1, verbose=1)
tuned_xgb.fit(X_train, y_train)

print(tuned_xgb.best_params_)
print(tuned_xgb.best_score_)

In [ ]:
model = tuned_xgb.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

In [ ]:
rfr_params = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'random_state': [32]
}
tuned_rfr = GridSearchCV(RandomForestRegressor(), rfr_params, cv=5, n_jobs=-1, verbose=1)
tuned_rfr.fit(X_train, y_train)

print(tuned_rfr.best_params_)
print(tuned_rfr.best_score_)

In [ ]:
model = tuned_rfr.best_estimator_
y_pred = model.predict(X_test)
print(f'RMSE: {RMSE(y_test, y_pred)}')
print(f'MSE: {MSE(y_test, y_pred)}')
print(f'MAE: {MAE(y_test, y_pred)}')

In [ ]:
model = tuned_svr.best_estimator_
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)
print(f'RMSE test: {RMSE(y_test, y_pred_test)}')
print(f'MSE test: {MSE(y_test, y_pred_test)}') 
print(f'MAE test: {MAE(y_test, y_pred_test)}')
print(f'RMSE train: {RMSE(y_train, y_pred_train)}')
print(f'MSE train: {MSE(y_train, y_pred_train)}')
print(f'MAE train: {MAE(y_train, y_pred_train)}')

In [ ]:
y_pred = np.concatenate((y_pred_train, y_pred_test))
y_true = np.concatenate((y_train, y_test))
print(f'RMSE all: {RMSE(y_true, y_pred)}')
print(f'MSE all: {MSE(y_true, y_pred)}')
print(f'MAE all: {MAE(y_true, y_pred)}')

In [ ]:
df_corr = pd.read_csv('data/beirut-daily-precipitation.csv')
x = df_corr['value']
y_corr = model.predict(x.values.reshape(-1, 1))
df_corr['value'] = y_corr
df_corr.to_csv('./data/beirut-daily-corrected.csv', index=False)

In [ ]:
pd.DataFrame({'date': dates, 'value_acc_new': y_pred, 'value_met': y_true}).to_csv('data/beirut-daily-comp.csv', index=False)